# 신경 신호 데이터 탐색 및 전처리

이 노트북은 적응형 신경 전기자극 시스템에서 사용되는 신경 신호 데이터의 탐색 및 전처리 과정을 설명합니다. 여러 유형의 신경 신호 데이터를 로드하고, 기본적인 분석과 시각화를 수행하며, 모델 학습을 위한 전처리 단계를 시연합니다.

## 1. 필요한 라이브러리 가져오기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal as signal
from scipy.fft import fft, fftfreq
import os
import h5py
from sklearn.preprocessing import StandardScaler

# 경고 메시지 숨기기
import warnings
warnings.filterwarnings('ignore')

# 그래프 스타일 설정
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

## 2. 데이터 로드 및 탐색

이 섹션에서는 다양한 형식(CSV, NPY, HDF5)의 신경 신호 데이터를 로드하고 기본 특성을 살펴봅니다.

In [ ]:
# 데이터 폴더 경로 설정
data_path = "../data/samples/"

# 사용 가능한 데이터 파일 목록 확인
if os.path.exists(data_path):
    files = os.listdir(data_path)
    csv_files = [f for f in files if f.endswith('.csv')]
    npy_files = [f for f in files if f.endswith('.npy')]
    h5_files = [f for f in files if f.endswith('.h5') or f.endswith('.hdf5')]
    
    print(f"CSV 파일: {csv_files}")
    print(f"NPY 파일: {npy_files}")
    print(f"HDF5 파일: {h5_files}")
else:
    print(f"경로 {data_path}를 찾을 수 없습니다. 샘플 데이터를 먼저 생성해주세요.")
    # 샘플 폴더 생성
    os.makedirs(data_path, exist_ok=True)
    print(f"{data_path} 디렉토리가 생성되었습니다. 이후 단계에서 샘플 데이터를 생성합니다.")

### 2.1 샘플 데이터 생성 (필요시)

데이터 파일이 존재하지 않는 경우, 분석을 위한 합성 신경 신호 데이터를 생성합니다.

In [ ]:
def generate_sample_data():
    """
    다양한 유형의 신경 신호 데이터를 생성하고 저장합니다.
    """
    # 샘플링 설정
    sampling_rate = 1000  # Hz
    duration = 5.0        # 초
    time = np.arange(0, duration, 1/sampling_rate)
    samples = len(time)
    
    # 랜덤 시드 설정 (재현성)
    np.random.seed(42)
    
    # 1. 정상 신경 신호 생성
    normal_data = np.random.normal(0, 0.5, (samples, 4))
    
    # 스파이크 패턴 추가
    for channel in range(4):
        # 각 채널에 랜덤한 위치에 스파이크 추가
        spike_positions = np.random.choice(np.arange(100, samples-100), 20, replace=False)
        for pos in spike_positions:
            # 스파이크 모양: 빠른 상승, 느린 하강
            normal_data[pos:pos+5, channel] = np.array([3, 4, 2, 1, 0.5])
    
    # 2. 손상된 신경 신호 생성
    damaged_data = normal_data.copy()
    # 일부 채널의 특정 구간에서 활동 감소
    damaged_data[1000:3000, 1:3] *= 0.2
    
    # 3. 자극 반응 신호 생성
    stim_response = np.zeros((samples, 4))
    # 주기적 자극 패턴 (500ms 간격으로 자극)
    for i in range(500, samples, 1000):
        if i+100 <= samples:
            for ch in range(4):
                # 채널별 다른 반응 강도
                response_amp = np.random.uniform(1.5, 3.0)
                # 자극 직후 강한 반응, 점차 감소
                decay = np.exp(-np.arange(100)/30)
                stim_response[i:i+100, ch] = response_amp * decay
    
    # 4. 잡음이 많은 신호 생성
    noisy_data = normal_data.copy()
    # 가우시안 노이즈 추가
    noisy_data += np.random.normal(0, 1.0, noisy_data.shape)
    # 60Hz 전원 노이즈 추가
    power_noise = 0.5 * np.sin(2 * np.pi * 60 * time).reshape(-1, 1)
    noisy_data += power_noise
    
    # 5. 다채널 (8채널) 신경 신호 생성
    multichannel_data = np.random.normal(0, 0.4, (samples, 8))
    for channel in range(8):
        # 채널별 기저 변동 추가
        drift = 0.2 * np.sin(2 * np.pi * 0.1 * channel * time)
        multichannel_data[:, channel] += drift
        
        # 스파이크 추가 (채널별 다른 빈도)
        num_spikes = 10 + 5 * channel  # 채널 번호에 따라 스파이크 수 증가
        spike_positions = np.random.choice(np.arange(100, samples-100), num_spikes, replace=False)
        for pos in spike_positions:
            spike_amp = 2.0 + 0.2 * channel  # 채널별 다른 스파이크 진폭
            multichannel_data[pos:pos+5, channel] = spike_amp * np.array([0.5, 1, 0.8, 0.4, 0.2])
    
    # CSV 파일 저장
    def save_data_as_csv(data, filename, channels=None):
        if channels is None:
            channels = [f'channel_{i+1}' for i in range(data.shape[1])]
        df = pd.DataFrame(data, columns=channels)
        df.insert(0, 'time', time)  # 시간 열 추가
        df.to_csv(os.path.join(data_path, filename), index=False)
        print(f"{filename} 저장 완료")
    
    # NPY 파일 저장
    def save_data_as_npy(data, filename):
        np.save(os.path.join(data_path, filename), data)
        print(f"{filename} 저장 완료")
    
    # 모든 데이터 저장
    save_data_as_csv(normal_data, "normal_neural_signal.csv")
    save_data_as_csv(damaged_data, "damaged_neural_signal.csv")
    save_data_as_csv(stim_response, "stim_response_signal.csv")
    save_data_as_csv(noisy_data, "noisy_neural_signal.csv")
    save_data_as_csv(multichannel_data, "multichannel_neural_signal.csv")
    
    save_data_as_npy(normal_data, "normal_neural_signal.npy")
    save_data_as_npy(damaged_data, "damaged_neural_signal.npy")
    save_data_as_npy(stim_response, "stim_response_signal.npy")
    
    # 데이터와 시간 배열을 함께 저장 (HDF5 형식)
    with h5py.File(os.path.join(data_path, "neural_signals.h5"), 'w') as f:
        f.create_dataset('time', data=time)
        f.create_dataset('normal', data=normal_data)
        f.create_dataset('damaged', data=damaged_data)
        f.create_dataset('stim_response', data=stim_response)
        f.create_dataset('noisy', data=noisy_data)
        f.create_dataset('multichannel', data=multichannel_data)
    print("neural_signals.h5 저장 완료")
    
    return time, normal_data, damaged_data, stim_response, noisy_data, multichannel_data

# 데이터 파일이 없으면 샘플 데이터 생성
if not os.path.exists(os.path.join(data_path, "normal_neural_signal.csv")):
    print("샘플 데이터 생성 중...")
    time, normal_data, damaged_data, stim_response, noisy_data, multichannel_data = generate_sample_data()
    print("샘플 데이터 생성 완료!")

### 2.2 CSV 데이터 로드 및 시각화

In [ ]:
# CSV 파일 로드
try:
    normal_df = pd.read_csv(os.path.join(data_path, "normal_neural_signal.csv"))
    damaged_df = pd.read_csv(os.path.join(data_path, "damaged_neural_signal.csv"))
    stim_response_df = pd.read_csv(os.path.join(data_path, "stim_response_signal.csv"))
    noisy_df = pd.read_csv(os.path.join(data_path, "noisy_neural_signal.csv"))
    
    # 정상 신경 신호 데이터 정보 출력
    print("=== 정상 신경 신호 데이터 정보 ===")
    print(normal_df.info())
    print("\n처음 5행:")
    print(normal_df.head())
    print("\n기본 통계:")
    print(normal_df.describe())
except FileNotFoundError:
    print("CSV 파일을 찾을 수 없습니다. 먼저 샘플 데이터를 생성해주세요.")

In [ ]:
# 신경 신호 데이터 시각화
def plot_neural_signal(df, title, duration=1.0):
    """
    신경 신호 데이터를 시각화합니다.
    
    Parameters:
    -----------
    df : DataFrame
        시각화할 신경 신호 데이터 (time 열 포함)
    title : str
        플롯 제목
    duration : float
        표시할 시간 구간 (초)
    """
    # 모든 채널 열 선택 (time 열 제외)
    channel_cols = [col for col in df.columns if col != 'time']
    
    # 표시할 샘플 수 계산
    time_col = df['time']
    samples_per_second = int(1 / (time_col[1] - time_col[0]))
    display_samples = int(duration * samples_per_second)
    
    # 시각화
    plt.figure(figsize=(14, 8))
    for i, channel in enumerate(channel_cols):
        # 채널별 오프셋 추가하여 겹치지 않게 표시
        offset = i * 5
        plt.plot(df['time'][:display_samples], 
                 df[channel][:display_samples] + offset, 
                 label=channel)
    
    # 플롯 설정
    plt.title(f"{title} (처음 {duration}초)", fontsize=14)
    plt.xlabel('시간 (초)', fontsize=12)
    plt.ylabel('진폭 + 오프셋', fontsize=12)
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# 다양한 신경 신호 데이터 시각화
try:
    # 정상 신경 신호
    plot_neural_signal(normal_df, "정상 신경 신호", duration=1.0)
    
    # 손상된 신경 신호
    plot_neural_signal(damaged_df, "손상된 신경 신호", duration=1.0)
    
    # 자극 반응 신호
    plot_neural_signal(stim_response_df, "자극 반응 신호", duration=2.0)
    
    # 잡음이 많은 신호
    plot_neural_signal(noisy_df, "잡음이 많은 신호", duration=1.0)
except NameError:
    print("데이터가 로드되지 않았습니다.")

## 3. 신호 전처리

이 섹션에서는 신경 신호 데이터에 적용할 수 있는 다양한 전처리 기법을 시연합니다.

### 3.1 노이즈 필터링

In [ ]:
def filter_neural_signal(signal_data, fs=1000, lowcut=5, highcut=200, order=4):
    """
    신경 신호에 대역 통과 필터를 적용합니다.
    
    Parameters:
    -----------
    signal_data : array-like
        필터링할 신호 데이터
    fs : float
        샘플링 주파수 (Hz)
    lowcut : float
        저주파 차단 (Hz)
    highcut : float
        고주파 차단 (Hz)
    order : int
        필터 차수
        
    Returns:
    --------
    filtered_data : array-like
        필터링된 신호 데이터
    """
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype='band')
    filtered_data = signal.filtfilt(b, a, signal_data)
    return filtered_data

try:
    # 잡음이 많은 신호에 대해 필터링 적용
    noisy_signal = noisy_df['channel_1'].values
    filtered_signal = filter_neural_signal(noisy_signal, lowcut=10, highcut=150)
    
    # 60Hz 노치 필터 적용 (전원 노이즈 제거)
    fs = 1000  # 샘플링 주파수 (Hz)
    notch_freq = 60.0  # 제거할 주파수 (Hz)
    quality_factor = 30.0  # 필터 Q 인자
    b_notch, a_notch = signal.iirnotch(notch_freq, quality_factor, fs)
    notched_signal = signal.filtfilt(b_notch, a_notch, filtered_signal)
    
    # 원본 신호와 필터링된 신호 비교
    plt.figure(figsize=(14, 10))
    
    # 시간 도메인 비교
    plt.subplot(2, 1, 1)
    t = noisy_df['time'].values[:1000]  # 첫 1초 데이터
    plt.plot(t, noisy_signal[:1000], 'b-', alpha=0.5, label='원본 신호')
    plt.plot(t, filtered_signal[:1000], 'r-', alpha=0.7, label='대역 통과 필터링')
    plt.plot(t, notched_signal[:1000], 'g-', alpha=0.7, label='노치 필터링')
    plt.title('시간 도메인: 원본 vs 필터링된 신호', fontsize=14)
    plt.xlabel('시간 (초)', fontsize=12)
    plt.ylabel('진폭', fontsize=12)
    plt.legend()
    plt.grid(True)
    
    # 주파수 도메인 비교
    plt.subplot(2, 1, 2)
    
    def plot_spectrum(signal_data, fs, label):
        # 신호의 주파수 스펙트럼 계산
        n = len(signal_data)
        yf = np.abs(fft(signal_data)/n)
        xf = fftfreq(n, 1/fs)[:n//2]
        plt.plot(xf[:int(250/fs*n)], 2.0*yf[:int(250/fs*n)], label=label, alpha=0.7)
    
    plot_spectrum(noisy_signal, fs, '원본 신호')
    plot_spectrum(filtered_signal, fs, '대역 통과 필터링')
    plot_spectrum(notched_signal, fs, '노치 필터링')
    
    plt.title('주파수 도메인: 원본 vs 필터링된 신호', fontsize=14)
    plt.xlabel('주파수 (Hz)', fontsize=12)
    plt.ylabel('진폭', fontsize=12)
    plt.xlim(0, 250)  # 0-250Hz 범위만 표시
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
except NameError:
    print("데이터가 로드되지 않았습니다.")

### 3.2 스파이크 검출 및 특성 추출

In [ ]:
def detect_spikes(signal_data, threshold_factor=4.0, min_distance=10):
    """
    신호에서 스파이크를 검출합니다.
    
    Parameters:
    -----------
    signal_data : array-like
        스파이크를 검출할 신호 데이터
    threshold_factor : float
        신호 표준편차의 몇 배를 임계값으로 사용할지 결정
    min_distance : int
        검출된 스파이크 간 최소 샘플 거리
        
    Returns:
    --------
    spike_indices : array-like
        검출된 스파이크의 인덱스 배열
    """
    # 표준편차 기반 임계값 계산
    threshold = threshold_factor * np.std(signal_data)
    
    # 임계값을 초과하는 샘플 찾기
    above_threshold = np.where(signal_data > threshold)[0]
    
    # 연속된 샘플을 하나의 스파이크로 그룹화
    spike_indices = []
    if len(above_threshold) > 0:
        # 첫 번째 스파이크 추가
        spike_indices.append(above_threshold[0])
        
        # 최소 거리 조건을 만족하는 스파이크 추가
        for i in range(1, len(above_threshold)):
            if above_threshold[i] - spike_indices[-1] >= min_distance:
                spike_indices.append(above_threshold[i])
    
    return np.array(spike_indices)

def extract_spike_features(signal_data, spike_indices, window_size=20):
    """
    검출된 스파이크 주변 신호의 특성을 추출합니다.
    
    Parameters:
    -----------
    signal_data : array-like
        원본 신호 데이터
    spike_indices : array-like
        검출된 스파이크의 인덱스 배열
    window_size : int
        스파이크 특성 추출을 위한 윈도우 크기
        
    Returns:
    --------
    features_dict : dict
        스파이크별 특성 정보
    """
    features_dict = {
        'amplitude': [],       # 스파이크 진폭
        'width': [],           # 스파이크 폭
        'rise_time': [],       # 상승 시간
        'decay_time': [],      # 감쇠 시간
        'waveforms': []        # 스파이크 파형
    }
    
    for idx in spike_indices:
        # 스파이크 주변 윈도우 추출 (경계 검사)
        start_idx = max(0, idx - window_size//2)
        end_idx = min(len(signal_data), idx + window_size//2)
        
        if end_idx - start_idx < window_size:
            continue  # 윈도우 크기가 충분하지 않으면 건너뜀
            
        waveform = signal_data[start_idx:end_idx]
        features_dict['waveforms'].append(waveform)
        
        # 진폭 계산
        amplitude = np.max(waveform) - np.min(waveform)
        features_dict['amplitude'].append(amplitude)
        
        # 스파이크 폭 계산 (반높이 전폭)
        half_amp = np.max(waveform) / 2
        above_half = np.where(waveform > half_amp)[0]
        if len(above_half) > 0:
            width = above_half[-1] - above_half[0]
            features_dict['width'].append(width)
        else:
            features_dict['width'].append(0)
        
        # 상승 및 감쇠 시간 계산
        peak_idx = np.argmax(waveform)
        rise_time = peak_idx
        decay_time = len(waveform) - peak_idx - 1
        
        features_dict['rise_time'].append(rise_time)
        features_dict['decay_time'].append(decay_time)
    
    # 배열로 변환
    for key in features_dict:
        if key != 'waveforms':
            features_dict[key] = np.array(features_dict[key])
    
    return features_dict

try:
    # 정상 신경 신호에서 스파이크 검출
    signal_data = normal_df['channel_1'].values
    time_data = normal_df['time'].values
    
    # 필터링 적용
    filtered_signal = filter_neural_signal(signal_data)
    
    # 스파이크 검출
    spike_indices = detect_spikes(filtered_signal, threshold_factor=3.0)
    
    # 스파이크 특성 추출
    features = extract_spike_features(filtered_signal, spike_indices)
    
    # 스파이크 검출 결과 시각화
    plt.figure(figsize=(14, 10))
    
    # 신호와 검출된 스파이크
    plt.subplot(2, 1, 1)
    display_samples = 1000  # 첫 1초 데이터
    plt.plot(time_data[:display_samples], filtered_signal[:display_samples], 'b-')
    
    # 검출된 스파이크 표시
    spike_mask = (spike_indices < display_samples)
    if np.any(spike_mask):
        displayed_spikes = spike_indices[spike_mask]
        plt.plot(time_data[displayed_spikes], filtered_signal[displayed_spikes], 'ro', markersize=8)
    
    plt.title('신경 신호와 검출된 스파이크', fontsize=14)
    plt.xlabel('시간 (초)', fontsize=12)
    plt.ylabel('진폭', fontsize=12)
    plt.grid(True)
    
    # 정규화된 스파이크 파형 중첩 표시
    plt.subplot(2, 1, 2)
    if len(features['waveforms']) > 0:
        # 최대 30개까지만 표시
        max_display = min(30, len(features['waveforms']))
        for i in range(max_display):
            waveform = features['waveforms'][i]
            # 진폭 정규화
            normalized = (waveform - np.min(waveform)) / (np.max(waveform) - np.min(waveform))
            plt.plot(np.arange(len(normalized)), normalized, 'b-', alpha=0.5)
        
        # 평균 파형 표시
        avg_waveform = np.mean(features['waveforms'][:max_display], axis=0)
        normalized_avg = (avg_waveform - np.min(avg_waveform)) / (np.max(avg_waveform) - np.min(avg_waveform))
        plt.plot(np.arange(len(normalized_avg)), normalized_avg, 'r-', linewidth=2, label='평균 파형')
        
        plt.title('정규화된 스파이크 파형', fontsize=14)
        plt.xlabel('샘플', fontsize=12)
        plt.ylabel('정규화된 진폭', fontsize=12)
        plt.legend()
        plt.grid(True)
    else:
        plt.text(0.5, 0.5, '검출된 스파이크 없음', horizontalalignment='center', verticalalignment='center', fontsize=14)
    
    plt.tight_layout()
    plt.show()
    
    # 스파이크 특성 통계
    if len(features['amplitude']) > 0:
        print(f"검출된 스파이크 수: {len(features['amplitude'])}")
        print(f"평균 스파이크 진폭: {np.mean(features['amplitude']):.3f} ± {np.std(features['amplitude']):.3f}")
        print(f"평균 스파이크 폭: {np.mean(features['width']):.3f} ± {np.std(features['width']):.3f} 샘플")
        print(f"평균 상승 시간: {np.mean(features['rise_time']):.3f} ± {np.std(features['rise_time']):.3f} 샘플")
        print(f"평균 감쇠 시간: {np.mean(features['decay_time']):.3f} ± {np.std(features['decay_time']):.3f} 샘플")
        
        # 스파이크 발화율 계산 (1초 단위 윈도우)
        fs = 1000  # 샘플링 주파수 (Hz)
        firing_rate = len(features['amplitude']) / (len(signal_data) / fs)
        print(f"평균 발화율: {firing_rate:.2f} Hz")
    else:
        print("검출된 스파이크가 없습니다.")
except NameError:
    print("데이터가 로드되지 않았습니다.")

### 3.3 시계열 데이터 준비 및 정규화

In [ ]:
def prepare_sequences(data, sequence_length=50, overlap=0, step=1):
    """
    신경 신호 데이터를 시퀀스로 변환합니다.
    
    Parameters:
    -----------
    data : array-like, shape (samples, features)
        시퀀스로 변환할 데이터
    sequence_length : int
        각 시퀀스의 길이
    overlap : int
        연속된 시퀀스 간 중첩 샘플 수
    step : int
        시퀀스 시작점 간 단계 크기
        
    Returns:
    --------
    sequences : array-like, shape (n_sequences, sequence_length, features)
        생성된 시퀀스 배열
    """
    n_samples, n_features = data.shape
    
    effective_step = sequence_length - overlap
    if effective_step <= 0:
        raise ValueError("overlap이 sequence_length보다 크거나 같을 수 없습니다.")
    
    # 시퀀스 시작점 계산
    start_indices = list(range(0, n_samples - sequence_length + 1, step))
    n_sequences = len(start_indices)
    
    # 결과 배열 초기화
    sequences = np.zeros((n_sequences, sequence_length, n_features))
    
    # 시퀀스 추출
    for i, start_idx in enumerate(start_indices):
        end_idx = start_idx + sequence_length
        sequences[i] = data[start_idx:end_idx]
    
    return sequences

try:
    # 신경 신호 데이터 준비
    # 정상 신호와 손상된 신호 모두 활용
    normal_signals = normal_df.iloc[:, 1:].values  # time 열 제외
    damaged_signals = damaged_df.iloc[:, 1:].values  # time 열 제외
    
    # 데이터 정규화
    scaler = StandardScaler()
    normal_scaled = scaler.fit_transform(normal_signals)
    damaged_scaled = scaler.transform(damaged_signals)  # 같은 스케일러 사용
    
    # 시퀀스 준비
    sequence_length = 100  # 100ms (1000Hz 가정)
    normal_sequences = prepare_sequences(normal_scaled, sequence_length=sequence_length, step=50)
    damaged_sequences = prepare_sequences(damaged_scaled, sequence_length=sequence_length, step=50)
    
    print(f"정상 신호 시퀀스 형태: {normal_sequences.shape}")
    print(f"손상된 신호 시퀀스 형태: {damaged_sequences.shape}")
    
    # 시퀀스 시각화
    plt.figure(figsize=(14, 10))
    
    # 정상 신호 시퀀스 샘플
    plt.subplot(2, 1, 1)
    for i in range(min(5, normal_sequences.shape[0])):
        for j in range(normal_sequences.shape[2]):  # 모든 채널 표시
            plt.plot(np.arange(sequence_length)/1000, normal_sequences[i, :, j] + j*2, alpha=0.7)
    
    plt.title('정상 신경 신호 시퀀스 샘플', fontsize=14)
    plt.xlabel('시간 (초)', fontsize=12)
    plt.ylabel('정규화된 진폭 + 오프셋', fontsize=12)
    plt.grid(True)
    
    # 손상된 신호 시퀀스 샘플
    plt.subplot(2, 1, 2)
    for i in range(min(5, damaged_sequences.shape[0])):
        for j in range(damaged_sequences.shape[2]):  # 모든 채널 표시
            plt.plot(np.arange(sequence_length)/1000, damaged_sequences[i, :, j] + j*2, alpha=0.7)
    
    plt.title('손상된 신경 신호 시퀀스 샘플', fontsize=14)
    plt.xlabel('시간 (초)', fontsize=12)
    plt.ylabel('정규화된 진폭 + 오프셋', fontsize=12)
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # 라벨 생성 (정상=1, 손상=0)
    y_normal = np.ones(normal_sequences.shape[0])
    y_damaged = np.zeros(damaged_sequences.shape[0])
    
    # 학습/테스트 데이터셋 생성 예제
    X = np.vstack([normal_sequences, damaged_sequences])
    y = np.hstack([y_normal, y_damaged])
    
    print(f"전체 데이터셋 형태: X.shape={X.shape}, y.shape={y.shape}")
    print(f"클래스 분포: 정상={np.sum(y==1)}, 손상={np.sum(y==0)}")
except NameError:
    print("데이터가 로드되지 않았습니다.")

## 4. 주파수 영역 분석

신경 신호의 주파수 특성을 분석합니다.

In [ ]:
def compute_spectrogram(signal_data, fs=1000, nperseg=256, noverlap=128):
    """
    신호의 스펙트로그램을 계산합니다.
    
    Parameters:
    -----------
    signal_data : array-like
        스펙트로그램을 계산할 신호 데이터
    fs : float
        샘플링 주파수 (Hz)
    nperseg : int
        각 세그먼트의 길이
    noverlap : int
        연속된 세그먼트 간 중첩 샘플 수
        
    Returns:
    --------
    f : array-like
        주파수 축 값
    t : array-like
        시간 축 값
    Sxx : array-like
        스펙트로그램 (파워 밀도)
    """
    f, t, Sxx = signal.spectrogram(signal_data, fs=fs, nperseg=nperseg, noverlap=noverlap)
    return f, t, Sxx

def compute_psd(signal_data, fs=1000, nperseg=1024):
    """
    신호의 파워 스펙트럼 밀도(PSD)를 계산합니다.
    
    Parameters:
    -----------
    signal_data : array-like
        PSD를 계산할 신호 데이터
    fs : float
        샘플링 주파수 (Hz)
    nperseg : int
        각 세그먼트의 길이
        
    Returns:
    --------
    f : array-like
        주파수 축 값
    Pxx : array-like
        파워 스펙트럼 밀도
    """
    f, Pxx = signal.welch(signal_data, fs=fs, nperseg=nperseg)
    return f, Pxx

try:
    # 신경 신호 유형별 주파수 분석
    fs = 1000  # 샘플링 주파수 (Hz)
    
    # 비교할 신호 선택
    normal_signal = normal_df['channel_1'].values
    damaged_signal = damaged_df['channel_1'].values
    stim_signal = stim_response_df['channel_1'].values
    noisy_signal = noisy_df['channel_1'].values
    
    # 필터링 적용
    filtered_normal = filter_neural_signal(normal_signal)
    filtered_damaged = filter_neural_signal(damaged_signal)
    filtered_stim = filter_neural_signal(stim_signal)
    filtered_noisy = filter_neural_signal(noisy_signal)
    
    # 파워 스펙트럼 밀도 계산
    f_normal, Pxx_normal = compute_psd(filtered_normal, fs=fs)
    f_damaged, Pxx_damaged = compute_psd(filtered_damaged, fs=fs)
    f_stim, Pxx_stim = compute_psd(filtered_stim, fs=fs)
    f_noisy, Pxx_noisy = compute_psd(filtered_noisy, fs=fs)
    
    # PSD 비교 시각화
    plt.figure(figsize=(14, 6))
    plt.semilogy(f_normal, Pxx_normal, 'b-', label='정상 신호', alpha=0.7)
    plt.semilogy(f_damaged, Pxx_damaged, 'r-', label='손상된 신호', alpha=0.7)
    plt.semilogy(f_stim, Pxx_stim, 'g-', label='자극 반응 신호', alpha=0.7)
    plt.semilogy(f_noisy, Pxx_noisy, 'k-', label='잡음이 많은 신호', alpha=0.7)
    
    plt.title('신경 신호 유형별 파워 스펙트럼 밀도 비교', fontsize=14)
    plt.xlabel('주파수 (Hz)', fontsize=12)
    plt.ylabel('파워 스펙트럼 밀도 (dB/Hz)', fontsize=12)
    plt.xlim(0, 200)  # 0-200Hz 범위만 표시
    plt.grid(True, which='both', linestyle='--', alpha=0.6)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # 스펙트로그램 계산 및 시각화
    plt.figure(figsize=(14, 12))
    
    # 정상 신호 스펙트로그램
    plt.subplot(2, 2, 1)
    f, t, Sxx = compute_spectrogram(filtered_normal[:3000], fs=fs)  # 3초 데이터
    plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud', cmap='viridis')
    plt.title('정상 신경 신호 스펙트로그램', fontsize=12)
    plt.xlabel('시간 (초)', fontsize=10)
    plt.ylabel('주파수 (Hz)', fontsize=10)
    plt.ylim(0, 200)  # 0-200Hz 범위만 표시
    plt.colorbar(label='파워 (dB)')
    
    # 손상된 신호 스펙트로그램
    plt.subplot(2, 2, 2)
    f, t, Sxx = compute_spectrogram(filtered_damaged[:3000], fs=fs)  # 3초 데이터
    plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud', cmap='viridis')
    plt.title('손상된 신경 신호 스펙트로그램', fontsize=12)
    plt.xlabel('시간 (초)', fontsize=10)
    plt.ylabel('주파수 (Hz)', fontsize=10)
    plt.ylim(0, 200)  # 0-200Hz 범위만 표시
    plt.colorbar(label='파워 (dB)')
    
    # 자극 반응 신호 스펙트로그램
    plt.subplot(2, 2, 3)
    f, t, Sxx = compute_spectrogram(filtered_stim[:3000], fs=fs)  # 3초 데이터
    plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud', cmap='viridis')
    plt.title('자극 반응 신호 스펙트로그램', fontsize=12)
    plt.xlabel('시간 (초)', fontsize=10)
    plt.ylabel('주파수 (Hz)', fontsize=10)
    plt.ylim(0, 200)  # 0-200Hz 범위만 표시
    plt.colorbar(label='파워 (dB)')
    
    # 잡음이 많은 신호 스펙트로그램
    plt.subplot(2, 2, 4)
    f, t, Sxx = compute_spectrogram(filtered_noisy[:3000], fs=fs)  # 3초 데이터
    plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud', cmap='viridis')
    plt.title('잡음이 많은 신호 스펙트로그램', fontsize=12)
    plt.xlabel('시간 (초)', fontsize=10)
    plt.ylabel('주파수 (Hz)', fontsize=10)
    plt.ylim(0, 200)  # 0-200Hz 범위만 표시
    plt.colorbar(label='파워 (dB)')
    
    plt.tight_layout()
    plt.show()
except NameError:
    print("데이터가 로드되지 않았습니다.")

## 5. 결론 및 요약

이 노트북에서는 다양한 유형의 신경 신호 데이터를 생성, 탐색, 분석하고 전처리하는 방법을 살펴보았습니다. 주요 내용은 다음과 같습니다:

1. **데이터 생성 및 탐색**: 여러 유형의 신경 신호 데이터(정상, 손상, 자극 반응, 잡음이 많은)를 생성하고 기본 특성을 탐색했습니다.

2. **신호 전처리**: 노이즈 필터링, 스파이크 검출, 특성 추출 등의 전처리 기법을 적용하여 신경 신호의 유의미한 정보를 추출했습니다.

3. **시계열 데이터 준비**: 딥러닝 모델 학습을 위한 시퀀스 데이터 생성 및 정규화 방법을 알아보았습니다.

4. **주파수 영역 분석**: PSD 및 스펙트로그램을 통해 신경 신호의 주파수 특성을 분석했습니다.

이러한 방법론은 적응형 신경 전기자극 시스템에서 신경 신호를 분석하고 자극 효과를 평가하는 데 중요한 기반이 됩니다. 처리된 데이터는 강화학습 에이전트나 LSTM과 같은 딥러닝 모델을 학습시키는 데 사용될 수 있으며, 이를 통해 개인별 최적화된 자극 프로토콜을 개발할 수 있습니다.